parts 1-3 (Yisroel Rosenblat) <br>
parts 4-5 (Massiel Sanchez)

resources:
- https://stackoverflow.com/questions/10610131/checking-if-a-field-contains-a-string
- https://stackoverflow.com/questions/26720050/how-to-count-the-number-of-documents-in-a-mongodb-collection
- https://stackoverflow.com/questions/9822575/how-to-delete-a-mongodb-collection-in-pymongo
- https://pymongo.readthedocs.io/en/stable/tutorial.html
 

Questions asked chat gpt:
- inspections.insert_one(document) ```WriteError: _id fields may not contain '$'-prefixed fields: $oid is not valid for storage., full error: {'index': 0, 'code': 52, 'errmsg': "_id fields may not contain '$'-prefixed fields: $oid is not valid for storage."}```






## setup db and imports

In [1]:
import pymongo
import json
from bson import ObjectId

In [2]:

myclient = pymongo.MongoClient("mongodb://localhost:27017/")

mydb = myclient["Violations"]


## step one 
### insert every object in file into a collection

In [3]:


inspections = mydb['inspections']

# delete db so could rerun easily
inspections.drop()


with open("city_inspections.json",'r') as f:
    for line in f:
        document = json.loads(line)
        # didnt work because the $ isn't allowed this is a quick fix
        document['_id'] = ObjectId(document['_id']['$oid'])
        inspections.insert_one(document)
        
print("loaded file")

loaded file


## step 2
### count the total amount for all time, 2015 and 2016

Total Incidents: 81047

2015 Incidents: 76383

2016 Incidents: 4664


In [4]:
def getCount(year = None) -> int:
    "get the count for a specific year or of everything"
    if year:
        return inspections.count_documents({'date':{'$regex':str(year)}})
    else:
        return inspections.count_documents({})
    

In [5]:
print(f"Total Incidents: {getCount()}")
print(f"2015 Incidents: {getCount(2015)}")
print(f"2016 Incidents: {getCount(2016)}")

Total Incidents: 81047
2015 Incidents: 76383
2016 Incidents: 4664


## Step 3 
### Find business violations for a certain business

In [10]:
business_name = input("enter a business name")
print("Business found" if inspections.find_one({"business_name":business_name}) else "Business Not found")

Business found


## Step 4
### Violations in Brooklyn and The Bronx

In [11]:
brooklyn_violations = {"address.city": "BROOKLYN"}
bronx_violations = {"address.city": "BRONX"}

# Count violations for Brooklyn and Bronx
brooklyn_total = inspections.count_documents(brooklyn_violations)
bronx_total = inspections.count_documents(bronx_violations)


# Print total violations in Brooklyn and Bronx
print("Total violations in Brooklyn:", brooklyn_total)
print("Total violations in The Bronx:", bronx_total)
print()

#Calculate the Sum
total_total = brooklyn_total + bronx_total
print("Total:", total_total)

#Calculate the difference
if brooklyn_total > bronx_total:
    difference = brooklyn_total - bronx_total
else:
    difference = bronx_total - brooklyn_total

print("Difference:", difference)
print()


#Print 5 businessess for each City
brooklyn_businesses = inspections.find(brooklyn_violations).limit(5)
bronx_businesses = inspections.find(bronx_violations).limit(5)

print("First five businesses in Brooklyn:")
for i, business in enumerate(brooklyn_businesses, start=1):
    print(f"{i}. {business['business_name']}")
    print("{number} {street}, {city}, {zip}".format(**business['address']))
    print()

print("First five businesses in Bronx:")
for i, business in enumerate(bronx_businesses, start=1):
    print(f"{i}. {business['business_name']}")
    print("{number} {street}, {city}, {zip}".format(**business['address']))
    print()

Total violations in Brooklyn: 26175
Total violations in The Bronx: 12329

Total: 38504
Difference: 13846

First five businesses in Brooklyn:
1. NEW FINEST BUILDERS INC
1320 E 55TH ST, BROOKLYN, 11234

2. NADER R. ABDELREHIM
51 NEWELL ST, BROOKLYN, 11222

3. AWNING SERVICES
450 PENNSYLVANIA AVE, BROOKLYN, 11207

4. QADIR AHMAD
1107 DORCHESTER RD, BROOKLYN, 11218

5. GREENPOINT BARGAINS, INC
211 MCGUINESS BLVD, BROOKLYN, 

First five businesses in Bronx:
1. LISANDRO CABRERA
2234 PALMER AVE, BRONX, 10475

2. THE WAY FOR VETERANS CONTACTING COMPANY LLC.
199 LINCOLN AVE, BRONX, 10454

3. BABUL MOHAMMED
1155 WALTON AVE, BRONX, 10452

4. GREGORY WHITE
1760 BRUCKNER BLVD, BRONX, 10473

5. BABUL MOHAMMED
1155 WALTON AVE, BRONX, 10452



## Step 5
### Print information based on given zip-code

In [12]:
import random
zip_code = input("Please Enter Zip Code to Search: ")

if len(zip_code) != 5 or not zip_code.isdigit():
    print("Zip-Code Not Found")


zip_code_int = int(zip_code)

#Print total violations in that Zip-Code
total = inspections.count_documents({"address.zip": zip_code_int})
print("The total number of business in", zip_code, ":", total)

#print 5 random businesses
business_list = list(inspections.find({"address.zip": zip_code_int}))

if len(business_list) < 5:
    print("Less than 5 businesses found in this zip code:")
    for i, business in enumerate(business_list, start=1):
        print(f"{i}. {business['business_name']}")

else:
    # Select 5 random businesses
    random_businesses = random.sample(business_list, 5)
    print("5 Randomly selected businesses:")
    for i, business in enumerate(random_businesses, start=1):
        print(f"{i}. {business['business_name']}")

The total number of business in 10452 : 622
5 Randomly selected businesses:
1. TWO STARS EXPRESS GROCERY INC.
2. JACKELINNE MINI MARKET CORP
3. ANA DELI GROCERY CORP.
4. COLORINO ROSARIO
5. DESTINY DELI GROCERY CORP.
